# Extraction of Basic Features from PPG Signal acquired from Empatica

## Import packages

In [ ]:
from utils.process_ppg import get_clean_segment, get_ppg_measures_batch
from scipy import signal
from datetime import datetime
%matplotlib inline

import os
import csv
import numpy as np
import matplotlib.pyplot as plt
import heartpy as hp
# import pandas as pd
# pd.set_option("display.precision", 2)
from scipy.interpolate import interp1d

# Load the data 
### Specify directory paths and the file containing PPG signal

In [ ]:
basepath = os.path.join('data', 'PulseSensor_Data')

# Change this to analyze PPG singal of your interest
filepath = os.path.join(basepath, r'raw_signal.npy')
sample_rate = 100  #Change this as per the sampling rate.
org_signal = np.load(filepath)

In [ ]:
start_sec = 0
end_sec = -1
raw_signal = org_signal[start_sec*sample_rate: end_sec*sample_rate]
time_array = np.arange(0, len(raw_signal)/sample_rate, 1/sample_rate)
plt.plot(time_array, raw_signal)

## Visualize raw signal

## Filter the signal and Plot


In [ ]:
sos = signal.butter(2, (1.0, 3.0), 'bandpass', fs=sample_rate, output='sos')
filtered = signal.sosfilt(sos, raw_signal)
# filtered = raw_signal

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(time_array, filtered)
plt.xlabel('Time(seconds)')
plt.title('Filtered PPG Signal')

In [ ]:
# This will try to get maximum clean segment available from the entire signal. With higher number for std_n, larger segment can be expected - though having more noise
filtered_clean = get_clean_segment(filtered, std_n=2.0)
plt.figure(figsize=(12, 4))
filt_time_array = np.arange(0, len(filtered_clean)/sample_rate, 1/sample_rate)
plt.plot(filt_time_array, filtered_clean)
plt.title('Clean Segment of Filtered PPG Signal')


## Compute the features using 'heartpy' Python package <br>
Plot the signal with peaks identified and print the measures derived

In [ ]:
wd, m = hp.process(filtered_clean, sample_rate=sample_rate)
print(wd.keys())

#display computed features
for measure in m.keys():
    print(measure, m[measure])

### Plot PPG signal with detected and rejected peaks

In [ ]:
#Plot PPG Signal with Peaks
plt.figure(figsize=(12,4))
x_axis = [i/sample_rate for i in range(len(wd['hr']))]
plt.plot(x_axis, wd['hr'])
plt.plot(np.array(wd['peaklist'])/sample_rate, wd['hr'][wd['peaklist']], '.', color='g', label='Peaks')
plt.plot(np.array(wd['removed_beats'])/sample_rate, wd['hr'][wd['removed_beats']], '.', color='r', label='Rejected Peaks')
mx = max(x_axis)
my = max(wd['hr'])
plt.text(0.1*mx,  my, 'HR: ' + str(int(round(m['bpm']))))
plt.text(0.3*mx, my, 'IBI: ' +  str(int(round(m['ibi']))))
plt.text(0.5*mx, my, 'SDNN: ' +  str(int(round(m['sdnn']))))
plt.text(0.7*mx, my, 'Resp: ' +  str(int(round(m['breathingrate'] * 60))))

plt.legend()
plt.xlabel('Time(seconds)')
plt.title('Filtered PPG Signal with Detected Peaks: ')

### Plot Breathing signal derived from PPG signal

In [ ]:
# Plot breathing signal derived from PPG signal
x_axis_br = [i/1000.0 for i in range(len(wd['breathing_signal']))]
plt.plot(x_axis_br, wd['breathing_signal'])
plt.xlabel('Time(seconds)')
plt.title('Breathing Signal')